In [10]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py

--2025-06-11 16:26:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-11 16:26:28 (64.3 MB/s) - ‘minsearch.py.1’ saved [5820/5820]



In [11]:
import minsearch

In [12]:
import json

In [13]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents = []
for course_dict in docs_raw :
    for doc in course_dict['documents'] :
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [17]:
index.fit(documents)

In [18]:
from openai import OpenAI

In [19]:
client = OpenAI()

In [21]:
def search(query) :
    boost = {'question':3.0, 'section': 0.5}

    results =index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results
    

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEST when answring the QUESTION
    
    QUESTION: {question}
    
    CONTEXT :
    {context}
    """.strip()

    context = ""

    for doc in search_results :
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content" :  prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
def rag(query)  :
    query = query
    results = search(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer

In [25]:
rag("The course just started, can I still enroll the course ?")

'Yes, even if the course has already started, you can still enroll and submit homeworks. However, be mindful of the deadlines for submitting the final projects, so try not to leave everything until the last minute.'

In [26]:
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch('http://localhost:9200')

In [28]:
index_settings =  {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/v3KJd8P7SPuHYiPSABAjMQ] already exists')

In [29]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents): 
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [32]:
query = 'I just discovered the course and it is already started, can I stil enrrol it?'

In [44]:
def elastic_search(query) :
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    result_docs = []
    for hit in response['hits']['hits'] :
        result_docs.append(hit['_source'])
    return  result_docs
    

In [45]:
def rag(query)  :
    query = query
    results = elastic_search(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer

In [46]:
rag(query)

"Yes, you can still enroll in the course after it has started. Even if you're not registered, you're eligible to submit the homeworks. Keep in mind that there will be deadlines for turning in the final projects, so make sure to manage your time accordingly."